In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import os
import glob
from datetime import datetime

!pip freeze > requirements.txt


: 

In [10]:
import sys
# import tensorflow.keras
import pandas as pd
# import sklearn as sk
import tensorflow as tf
# print(f"Tensor Flow Version: {tf.__version__}")
# print(f"Keras Version: {tensorflow.keras.__version__}")
# print()
# print(f"Python {sys.version}")
# print(f"Pandas {pd.__version__}")
# print(f"Scikit-Learn {sk.__version__}")
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(logical_gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
def app_log(nv,msg):
    print(" "*nv + "[{}] INFO - {}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"),msg))
    

In [4]:

app_log(0,"Limpando ambiente")

if not os.path.exists("database\db"):
    os.makedirs("database\db")

if os.path.exists("database\db\dbo.db"):
  os.remove("database\db\dbo.db")

app_log(0,"Criando banco de dados")
conn = sqlite3.connect("database\db\dbo.db")
conn.execute("ATTACH DATABASE 'database\db\dbo.db' AS 'dbo';")
cur  = conn.cursor()

app_log(0,"Criando tabelas no banco de dados")
sql_file = open(r"database\ddl\create_tables.sql",encoding="ISO-8859-1")
sql_as_string = sql_file.read()
sql_file.close()
cur.executescript(sql_as_string)

conn.commit()
conn.close()


[31/12/2023 11:39:22] INFO - Limpando ambiente
[31/12/2023 11:39:22] INFO - Criando banco de dados
[31/12/2023 11:39:22] INFO - Criando tabelas no banco de dados


In [5]:
conn = sqlite3.connect("database\db\dbo.db")
dir_list = glob.glob('data/*.csv')
app_log(0,"Carga de dados no banco de dados")
app_log(1,"Tabelas Dimensao ")
for arq in filter(lambda file: "Dim" in file,dir_list):
    cursor = conn.execute("select * from {}".format(arq[5:-4]))
    colunas = [description[0] for description in cursor.description]
    df = pd.read_csv(arq,sep="|",
                     encoding="utf16",header=None,names=colunas,lineterminator ='\n',
                     keep_default_na=False)
    df = df.replace(r'\r',' ', regex=True) 
    app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
    df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

app_log(1,"Tabelas Fato")
for arq in filter(lambda file: "Fact" in file,dir_list):
    cursor = conn.execute("select * from {}".format(arq[5:-4]))
    colunas = [description[0] for description in cursor.description]
    df = pd.read_csv(arq,sep="|",
                     encoding="utf16",header=None,names=colunas,lineterminator ='\n',
                     keep_default_na=False)
    df = df.replace(r'\r',' ', regex=True) 
    app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
    df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

app_log(1,"Demais Tabelas ")
for arq in filter(lambda file: "Fact" not in file and "Dim" not in file ,dir_list):
    try:
        cursor = conn.execute("select * from {}".format(arq[5:-4]))
        colunas = [description[0] for description in cursor.description]
        df = pd.read_csv(arq,sep="|",
                        encoding="utf16",header=None,names=colunas,lineterminator ='\n',
                        keep_default_na=False)
        df = df.replace(r'\r',' ', regex=True) 
        app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
        df.to_sql(arq[5:-4],conn,if_exists='append',index=False)
    except:
        df = pd.read_csv(arq,sep="|",
                        encoding="utf16",header=None,lineterminator ='\n',
                        keep_default_na=False)
        df = df.replace(r'\r',' ', regex=True) 
        app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
        df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

conn.close()

[31/12/2023 11:39:23] INFO - Carga de dados no banco de dados
 [31/12/2023 11:39:23] INFO - Tabelas Dimensao 
  [31/12/2023 11:39:23] INFO - Carregando DimAccount - 99 linhas
  [31/12/2023 11:39:23] INFO - Carregando DimCurrency - 105 linhas
  [31/12/2023 11:39:23] INFO - Carregando DimCustomer - 18484 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimDate - 3652 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimDepartmentGroup - 7 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimEmployee - 296 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimGeography - 655 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimOrganization - 14 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimProduct - 606 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimProductCategory - 4 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimProductSubcategory - 37 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimPromotion - 16 linhas
  [31/12/2023 11:39:24] INFO - Carregando DimReseller - 701 linhas


In [6]:
conn = sqlite3.connect("database\db\dbo.db")
sql_query = """SELECT * FROM FactInternetSales;"""
df = pd.read_sql(sql_query,conn)
conn.close()
df

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,...,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,,,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,,,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,,,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,,,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,,,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,485,20140128,20140209,20140204,15868,1,100,6,SO75122,1,...,8.2205,8.2205,21.9800,1.7584,0.5495,,,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00
60394,225,20140128,20140209,20140204,15868,1,100,6,SO75122,2,...,6.9223,6.9223,8.9900,0.7192,0.2248,,,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00
60395,485,20140128,20140209,20140204,18759,1,100,6,SO75123,1,...,8.2205,8.2205,21.9800,1.7584,0.5495,,,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00
60396,486,20140128,20140209,20140204,18759,1,100,6,SO75123,2,...,59.4660,59.4660,159.0000,12.7200,3.9750,,,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00
